In [10]:
pip install -U sentence-transformers



  Using cached sentence_transformers-3.3.0-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.3.0-py3-none-any.whl (268 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successfully uninstalled sentence-transformers-3.0.1



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from hide import gemini_api, hf_api as inference_api_key
import time
import os

c:\Users\WAHIDULLAH\Desktop\Error-404-ESummit-2024\law\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def create_db():
    embedding = HuggingFaceInferenceAPIEmbeddings(
        api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
    )
    vectorStore = FAISS.load_local("faiss_index_law", embedding, allow_dangerous_deserialization=True)
    return vectorStore

In [ ]:
def create_chain(vectorStore):
    model = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_api)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a friendly Law AI Bot. You always greet the user well and treat respectfully. You have knowledge related to Indian Constitution. You have to give detailed answer on any law for atleast 500 words. \n Note: Only respond as Law AI, no other character. ALWAYS START Your Answer Directly, without mentioning your name. Do Not add extra Spaces. Always write in third person. Always make your answer clear and complete. Answer only if a relevant question is asked, and Answer only if you know the answer, otherwise don't answer with unknown or vague information. Take care of the ethics. Answer the user's questions based on the relevant sentences: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ])

    chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )

    
    retriever = vectorStore.as_retriever(search_kwargs={"k": 10})

    retriever_prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above chat history, generate a search query to look up in order to get information relevant to the conversation")
    ])

    history_aware_retriever = create_history_aware_retriever(
        llm=model,
        retriever=retriever,
        prompt=retriever_prompt
    )

    retrieval_chain = create_retrieval_chain(
        
        history_aware_retriever,
        chain
    )

    return retrieval_chain

In [14]:
def process_chat(chain, question, chat_history):
    response = chain.invoke({
        "chat_history": chat_history,
        "input": question,
    })
    return response.get("answer")

In [ ]:
vectorStore = create_db()
chain = create_chain(vectorStore)


chat_history = []

while True:
    user_input = input("User: ")    
    if user_input.lower() == 'exit':
        break
    response = process_chat(chain, user_input, chat_history)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))
    print("Law AI: " + response)
    print("--------------------------------------------------------------------------------")   

Law AI: Article 370 of the Indian Constitution, as it existed before its revocation on August 5, 2019, provided special autonomous status to the State of Jammu and Kashmir.  The article's primary function was to grant temporary provisions regarding the state's integration into the Indian Union.  It allowed for a degree of self-governance while still maintaining its position within India.

The original Article 370, as incorporated in the Constitution, allowed for the application of specific provisions of the Indian Constitution to Jammu and Kashmir with modifications.  This meant that not all laws passed by the Indian Parliament automatically applied to the state.  The state had its own constitution and its own flag.

Clause (1) of Article 370 established that the provisions of the Indian Constitution, except for those specified in the article, would apply to Jammu and Kashmir subject to the modifications made by Presidential Orders issued on the recommendations of the State's Constitue